# Tool을 사용한 AgentCore Memory (장기) 기반 Strands Agent

## 개요
이 노트북은 Strands와 AgentCore Memory를 사용하여 대화형 AI Agent에 장기 메모리 기능을 구현하는 방법을 보여줍니다. 단기 상호작용에서 중요한 정보를 추출하고 통합하여 Agent가 시간이 지남에 따라 여러 대화 세션에서 핵심 세부 정보를 기억할 수 있도록 하는 방법을 배우게 됩니다.

## 튜토리얼 세부 정보
**사용 사례:** 메모리 지속성을 가진 요리 어시스턴트

| 정보         | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 장기 대화형                                                         |
| Agent 유형          | 요리 어시스턴트                                                               |
| Agentic Framework   | Strands Agent                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성 요소 | AgentCore 'User Preferences' Memory 추출, Memory 저장 및 검색을 위한 Memory Tool              |
| 예제 복잡도  | 초급                                                                     |

다음을 배우게 됩니다:
- 장기 보존을 위한 추출 전략으로 AgentCore Memory 구성
- 이전 대화 기록으로 메모리 채우기
- 장기 메모리를 사용하여 대화 세션 전반에 걸쳐 개인화된 경험 제공
- Strands Agent Framework를 AgentCore Memory Tool과 통합

## 시나리오 컨텍스트

이 튜토리얼에서는 고도로 개인화된 레스토랑 추천을 제공하도록 설계된 요리 어시스턴트의 역할을 맡게 됩니다. AgentCore Memory의 장기 보존 및 자동 정보 추출을 활용하여 Agent는 여러 대화에서 식단 선택 및 선호하는 요리와 같은 사용자 선호도를 기억할 수 있습니다. 이러한 메모리 지속성을 통해 Agent는 대화가 며칠 또는 몇 주에 걸쳐 진행되더라도 맞춤형 제안과 원활한 사용자 경험을 제공할 수 있습니다. 이 시나리오는 구조화된 메모리 구성과 구성 가능한 전략이 대화형 AI가 단기 기억을 넘어 진정으로 매력적이고 컨텍스트 인식 상호작용을 생성할 수 있도록 하는 방법을 보여줍니다.


## 아키텍처

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK
환경을 설정하고 적절한 추출 전략으로 장기 메모리 리소스를 생성하여 시작하겠습니다!

## Step 1: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작하겠습니다.

In [ ]:
# 필요한 패키지 설치 (-q: quiet mode, -r: requirements 파일 사용)
!pip install -qr requirements.txt

In [ ]:
import time
import logging
import time
from datetime import datetime

Amazon Bedrock Model 및 AgentCore에 대한 적절한 권한이 있는 리전과 역할을 정의합니다

In [ ]:
import os

# 로깅 설정: INFO 레벨로 시간, 레벨, 메시지 형식 지정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("culinary-memory")

# 환경 변수에서 AWS region 가져오기 (기본값: us-west-2)
region = os.getenv('AWS_REGION', 'us-west-2')

## Step 2: 장기 전략으로 Memory 생성

이 섹션에서는 장기 메모리 기능으로 구성된 메모리 리소스를 생성합니다. 이전의 단기 메모리 예제와 달리 이 구현에는 통합된 정보 보존을 가능하게 하는 특정 메모리 전략이 포함되어 있습니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# AgentCore Memory 클라이언트 초기화
client = MemoryClient(region_name=region)

memory_name = "CulinaryAssistant"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Long-Term Memory...")

    # We use a more descriptive name for our long-term memory resource
    memory_name = memory_name

    # USER_PREFERENCE 전략으로 장기 메모리 생성
    # - 대화에서 사용자 선호도를 자동으로 추출하여 저장
    # - event_expiry_days: 단기 이벤트는 7일 후 만료
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Culinary Assistant Agent with long term memory",
        strategies=[{
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "UserPreferences",
                        "description": "Captures user preferences",
                        "namespaces": ["user/{actorId}/preferences/"]  # 사용자별 선호도 저장 경로
                    }
                }],
        event_expiry_days=7,
        max_wait=300,  # 최대 300초 대기
        poll_interval=10  # 10초마다 상태 확인
    )

    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
    
except ClientError as e:
    # 이미 존재하는 메모리인 경우 기존 ID 사용
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # 에러 발생 시 생성된 메모리 정리
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 장기 Memory 전략 이해

이 메모리 생성의 주요 차이점은 **메모리 전략**의 추가입니다. 구성 요소를 분석해 보겠습니다:

#### 1. User Preference Memory 전략

이 전략은 대화에서 사용자 선호도를 자동으로 식별하고 추출합니다:

```python
"userPreferenceMemoryStrategy": {
    "name": "UserPreferences",
    "description": "Captures user preferences",
    "namespaces": ["user/{actorId}/preferences/"]
}
```

#### 2. Memory Namespace

`namespaces` 매개변수는 추출된 정보가 저장되는 위치를 정의합니다:

```python
"namespaces": ["user/{actorId}/preferences/"]
```

이 메모리 전략은 단순히 대화를 기억하는 것이 아니라 실제로 해당 대화 내의 중요한 정보를 이해하고 구성하여 향후 사용할 수 있도록 하는 보다 정교한 메모리 시스템을 생성합니다.

## Step 3: 이전 대화를 Memory에 저장

이 섹션에서는 단기 메모리를 채우는 방법을 보여주며, 이는 백그라운드에서 장기 메모리 추출 프로세스를 자동으로 트리거합니다.

### 단기 Memory 채우기

추출 전략으로 구성된 메모리 리소스에 대화를 저장하면 시스템은 추가 코드 없이 장기 보존을 위해 이 정보를 자동으로 처리합니다.

In [ ]:
# 고유한 사용자 및 세션 식별자 생성 (타임스탬프 기반)
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"foodie-{datetime.now().strftime('%Y%m%d%H%M%S')}"
# 사용자 선호도 저장을 위한 namespace 경로
namespace = f"user/{actor_id}/preferences/"

In [ ]:
# 이전 대화 내역 (사용자 선호도 포함)
# - 채식주의자, 두부와 신선한 채소 선호, 이탈리아 요리 선호 등의 정보 포함
previous_messages = [
    ("Hi, I'm John", "USER"),
    ("Hi John, how can I help you with food recommendations today?", "ASSISTANT"),
    ("I'm looking for some vegetarian dishes to try this weekend.", "USER"),
    ("That sounds great! I'd be happy to help with vegetarian recommendations. Do you have any specific ingredients or cuisine types you prefer?", "ASSISTANT"),
    ("Yes, I really like tofu and fresh vegetables in my dishes", "USER"),
    ("Perfect! Tofu and fresh vegetables make for excellent vegetarian meals. I can suggest some stir-fries, Buddha bowls, or tofu curries. Do you have any other preferences?", "ASSISTANT"),
    ("I also really enjoy Italian cuisine. I love pasta dishes and would like them to be vegetarian-friendly.", "USER"),
    ("Excellent! Italian cuisine has wonderful vegetarian options. I can recommend pasta primavera, mushroom risotto, eggplant parmesan, or penne arrabbiata. The combination of Italian flavors with vegetarian ingredients creates delicious meals!", "ASSISTANT"),
    ("I spent 2 hours looking through cookbooks but couldn't find inspiring vegetarian Italian recipes", "USER"),
    ("I'm sorry you had trouble finding inspiring recipes! Let me help you with some creative vegetarian Italian dishes. How about stuffed bell peppers with Italian herbs and rice, spinach and ricotta cannelloni, or a Mediterranean vegetable lasagna?", "ASSISTANT"),
    ("Hey, I appreciate food assistants with good taste", "USER"),
    ("Ha! I definitely try to bring good taste to the table! Speaking of which, shall we explore some more vegetarian Italian recipes that might inspire you?", "ASSISTANT")
]

In [ ]:
print("\nHydrating short term memory with previous conversations...")

# 대화 내역을 단기 메모리에 저장
# - 이 작업은 자동으로 장기 메모리 추출 프로세스를 트리거함
initial = client.create_event(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    messages=previous_messages,
)
print("✓ Conversation saved in short term memory")

대화 메시지가 포함된 이벤트가 올바르게 저장되었는지 확인하겠습니다.

In [ ]:
# 저장된 이벤트 목록 조회 (최근 5개)
events = client.list_events(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    max_results=5
)
events

이 셀은 실행 중에 정보 메시지를 표시하도록 로깅 시스템을 구성하여 코드가 실행될 때 무슨 일이 일어나고 있는지 추적하는 데 도움을 줍니다.

### 백그라운드에서 일어나는 일

`create_event` 호출 후 다음이 자동으로 발생합니다:

1. **단기 저장**: 전체 대화가 원시 형태로 저장됩니다
2. **추출 트리거**: 메모리 시스템은 이 메모리에 UserPreference 전략이 구성되어 있음을 감지합니다
3. **백그라운드 처리**: 추가 코드 없이 시스템은:
   - 선호도 지표에 대한 대화를 분석합니다
   - "I'm vegetarian" 및 "I really enjoy Italian cuisine"과 같은 진술을 식별합니다
   - 이러한 선호도를 구조화된 데이터로 추출합니다
4. **장기 통합**: 추출된 선호도는 구성된 namespace(`user/{actorId}/preferences/`)에 저장됩니다

추출 및 통합은 자동으로 발생합니다 - Agent와 대화를 유지하거나 단기 메모리를 채우기만 하면 되며, 메모리 생성 중에 구성한 전략이 나머지를 처리합니다.

이 자동 프로세스는 단기 대화 기록이 만료된 후에도 중요한 정보가 장기 메모리에 보존되도록 합니다.


## 장기 Memory 검색

이 섹션에서는 장기 메모리에 저장된 추출된 선호도에 액세스하는 방법을 살펴봅니다. 대화 턴에 초점을 맞춘 단기 메모리 검색과 달리 장기 메모리 검색은 추출되고 통합된 구조화된 정보에 액세스하는 데 초점을 맞춥니다.

### 장기 Memory에서 사용자 선호도 액세스

장기 메모리에서 정보를 검색하려면 메모리 생성 중에 정의된 namespace 구조를 사용합니다:


In [ ]:
# 메모리 추출 프로세스가 완료될 때까지 30초 대기
time.sleep(30)

try:
    # 장기 메모리에서 음식 선호도 검색
    food_preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=namespace,
        query="food preferences",
        top_k=3  # 가장 관련성 높은 3개 결과 반환
    )

    if food_preferences:
        print(f"Retrieved {len(food_preferences)} relevant preference records:")
        for i, record in enumerate(food_preferences):
            print(f"\nMemory {i+1}:")
            print(f"- Content: {record.get('content', 'Not specified')}")
    else:
        print("No matching preference records found.")

except Exception as e:
    print(f"Error retrieving preference records: {e}")

이 방법을 사용하면 필요할 때 관련 메모리를 검색할 수 있습니다. 이제 기본 사항을 배웠으니 Agent를 구축해 보겠습니다!

## Step 4: Agent 생성 
이 섹션에서는 네이티브 `agent_core_memory` Tool을 사용하여 AgentCore Memory를 Strands Agent와 통합하는 방법을 살펴봅니다.

#### 장기 Memory 기능을 갖춘 Agent 설정
메모리 지원 Agent를 생성하기 위해 Strands Framework를 사용하고 AgentCore Memory 리소스에 연결합니다

In [ ]:
from strands import tool, Agent
# AgentCore Memory Tool Provider import
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

In [ ]:
system_prompt = f"""You are the Culinary Assistant, a sophisticated restaurant recommendation assistant.

PURPOSE:
- Help users discover restaurants based on their preferences
- Remember user preferences throughout the conversation
- Provide personalized dining recommendations

You have access to a Memory tool that enables you to:
- Store user preferences (dietary restrictions, favorite cuisines, budget preferences, etc.)
- Retrieve previously stored information to personalize recommendations

"""

In [ ]:
# Memory Tool Provider 초기화
# - memory_id: 사용할 메모리 리소스 ID
# - actor_id, session_id: 사용자 및 세션 식별자
# - namespace: 메모리 저장/검색 경로
provider = AgentCoreMemoryToolProvider(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    namespace=namespace
)

# Memory Tool을 포함한 Strands Agent 생성
agent = Agent(tools=provider.tools, model="global.anthropic.claude-haiku-4-5-20251001-v1:0",system_prompt=system_prompt)

이미 단기 및 장기 메모리를 채웠으므로 Agent에서 직접 메모리를 검색해 보겠습니다!

In [ ]:
# Agent에게 저장된 음식 선호도를 기반으로 레스토랑 추천 요청
agent("Give me restaurant recommendations in Irvine based on my food preferences")

Agent는 사용자의 메모리를 검색하기 위해 retrieve_memory_records 메서드를 사용했어야 합니다.

훌륭합니다! 이제 AgentCore 장기 Memory에서 메모리를 검색할 수 있는 작동하는 Strands Agent를 갖게 되었습니다!

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 메모리를 삭제하겠습니다.

In [ ]:
# 메모리 리소스 삭제 (정리)
# 주석 처리되어 있음 - 필요시 주석 해제하여 실행
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)